In [16]:
import numpy as np
import pandas as pd

In [ ]:
with open('distances_mumbai.csv') as f:
    f.readlines()

In [141]:
#def som():
f = open('assets/locations_mumbai.csv')
cities = pd.read_csv(f)
city = cities[['Longitude', 'Latitude']]
city

,Longitude,Latitude
0,72.838945,18.935783
1,72.823287,18.932132
2,72.831722,18.958083
3,72.804173,18.964716
4,72.836387,18.997817
5,72.847510,19.017791
6,72.863948,19.015427


In [142]:
n = 7
eta = 1
radii = 1.5
iterations = 3

np.random.seed(1024)
network = np.random.rand(n,2)
print(network)
for i in range(iterations):
    winner  = euclidean_distance(network, city.sample(1).values).argmin()
    print(winner)
    gaussian= get_neighborhood(winner, int(radii), n)
    print(gaussian)
    network+= gaussian[:, np.newaxis]*eta*(city - network)
    print(network)
    eta = eta*0.99997
    radii = radii*0.9997

winner
route = get_route(city, network)
# city['winner'] = city.apply(lambda c: select_closest(network, c), axis=1, raw=True

# city.sort_values('winner').index
# city
route

[[0.64769123 0.99691358]
 [0.51880326 0.65811273]
 [0.59906347 0.75306733]
 [0.13624713 0.00411712]
 [0.14950888 0.698439  ]
 [0.59335256 0.89991535]
 [0.44445739 0.316785  ]]
0
[1.         0.60653066 0.13533528 0.011109   0.011109   0.13533528
 0.60653066]
   Longitude   Latitude
0  72.838945  18.935783
1  44.373689  11.741866
2  10.374691   3.216848
3   0.943515   0.214750
4   0.956987   0.901727
5  10.371889   3.351903
6  44.369099  11.658085
0
[1.         0.60653066 0.13533528 0.011109   0.011109   0.13533528
 0.60653066]
   Longitude   Latitude
0  72.838945  18.935783
1  61.628725  16.102852
2  18.827077   5.347129
3   1.741791   0.423037
4   1.755471   1.102750
5  18.826792   5.471987
6  61.651580  16.120404
0
[1.         0.60653066 0.13533528 0.011109   0.011109   0.13533528
 0.60653066]
   Longitude   Latitude
0  72.838945  18.935783
1  68.418163  17.818794
2  26.135373   7.189061
3   2.531175   0.629004
4   2.545061   1.301534
5  26.137262   7.305102
6  68.451817  17.876219


C:\Users\PranavMaheshwari\AppData\Local\Temp\ipykernel_14180\2094445227.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cities['winner'] = cities.apply(


Int64Index([0, 1, 2, 3, 4, 5, 6], dtype='int64')

In [70]:
network = np.random.rand(5, 2)
gaussian= get_neighborhood(1, 1, 5)
gaussian

array([0.60653066, 1.        , 0.60653066, 0.13533528, 0.13533528])

In [48]:
n = generate_network(5)

In [64]:
def get_neighborhood(center, radix, domain):

    # Impose an upper bound on the radix to prevent NaN and blocks
    if radix < 1:
        radix = 1

    # Compute the circular network distance to the center
    deltas = np.absolute(center - np.arange(domain))
    distances = np.minimum(deltas, domain - deltas)

    # Compute Gaussian distribution around the given center
    return np.exp(-(distances*distances) / (2*(radix*radix)))

In [133]:
def get_route(cities, network):
    """Return the route computed by a network."""
    cities['winner'] = cities.apply(
        lambda c: select_closest(network, c),
        axis=1, raw=True)

    return cities.sort_values('winner').index

In [ ]:
def normalize(points):
    """
    Return the normalized version of a given vector of points.

    For a given array of n-dimensions, normalize each dimension by removing the
    initial offset and normalizing the points in a proportional interval: [0,1]
    on y, maintining the original ratio on x.
    """
    ratio = (points.x.max() - points.x.min()) / (points.y.max() - points.y.min()), 1
    ratio = np.array(ratio) / max(ratio)
    norm = points.apply(lambda c: (c - c.min()) / (c.max() - c.min()))
    return norm.apply(lambda p: ratio * p, axis=1)


In [81]:
def euclidean_distance(a, b):
    """Return the array of distances of two numpy arrays of points."""
    return np.linalg.norm(a - b, axis=1)

def route_distance(cities):
    """Return the cost of traversing a route of cities in a certain order."""
    points = cities[['x', 'y']]
    distances = euclidean_distance(points, np.roll(points, 1, axis=0))
    return np.sum(distances)
